# Label Studio | Label Studio

> [Label Studio](https://labelstud.io/guide/get_started)は、大規模言語モデル（LLMs）のファインチューニング用データラベリングにおいて、LangChainに柔軟性を提供するオープンソースのデータラベリングプラットフォームです。また、カスタムトレーニングデータの準備や、人間のフィードバックを通じたレスポンスの収集と評価を可能にします。
>
> > [Label Studio](https://labelstud.io/guide/get_started) is an open-source data labeling platform that provides LangChain with flexibility when it comes to labeling data for fine-tuning large language models (LLMs). It also enables the preparation of custom training data and the collection and evaluation of responses through human feedback.

このガイドでは、LangChainパイプラインを`Label Studio`に接続する方法を学びます。

> In this guide, you will learn how to connect a LangChain pipeline to `Label Studio` to:

* すべての入力プロンプト、会話、および応答を一つの`Label Studio`プロジェクトで集約します。これにより、ラベリングと分析をより容易にするために、全てのデータを一箇所に統合します。

  > Aggregate all input prompts, conversations, and responses in a single `Label Studio` project. This consolidates all the data in one place for easier labeling and analysis.

* プロンプトとレスポンスを洗練させ、教師ありファインチューニング（SFT）および人間のフィードバックによる強化学習（RLHF）のシナリオ用のデータセットを作成します。ラベル付けされたデータは、LLMの性能を向上させるためのさらなるトレーニングに使用できます。

  > Refine prompts and responses to create a dataset for supervised fine-tuning (SFT) and reinforcement learning with human feedback (RLHF) scenarios. The labeled data can be used to further train the LLM to improve its performance.

* モデルの応答を人間のフィードバックで評価します。`Label Studio`は、人間がモデルの応答をレビューしてフィードバックを提供するためのインターフェースを提供し、評価と反復作業を可能にします。

  > Evaluate model responses through human feedback. `Label Studio` provides an interface for humans to review and provide feedback on model responses, allowing evaluation and iteration.





## Installation and setup | インストールとセットアップ




まず、Label StudioとLabel Studio APIクライアントの最新バージョンをインストールしてください：

> First install latest versions of Label Studio and Label Studio API client:




In [ ]:
!pip install -U label-studio label-studio-sdk openai

次に、コマンドラインで `label-studio` を実行して、`http://localhost:8080` でローカルのLabel Studioインスタンスを起動します。より多くのオプションについては、[Label Studioのインストールガイド](https://labelstud.io/guide/install)を参照してください。

> Next, run `label-studio` on the command line to start the local LabelStudio instance at `http://localhost:8080`. See the [Label Studio installation guide](https://labelstud.io/guide/install) for more options.




APIコールを行うにはトークンが必要です。

> You'll need a token to make API calls.

ブラウザでLabelStudioインスタンスを開き、「アカウント & 設定 > アクセストークン」に移動して、キーをコピーしてください。

> Open your LabelStudio instance in your browser, go to `Account & Settings > Access Token` and copy the key.

LabelStudioのURL、APIキー、およびOpenAI APIキーを環境変数として設定してください：

> Set environment variables with your LabelStudio URL, API key and OpenAI API key:




In [ ]:
import os

os.environ["LABEL_STUDIO_URL"] = "<YOUR-LABEL-STUDIO-URL>"  # e.g. http://localhost:8080
os.environ["LABEL_STUDIO_API_KEY"] = "<YOUR-LABEL-STUDIO-API-KEY>"
os.environ["OPENAI_API_KEY"] = "<YOUR-OPENAI-API-KEY>"

## Collecting LLMs prompts and responses | LLMのプロンプトとレスポンスの収集




ラベリングに使用されるデータはLabel Studio内のプロジェクトに保存されています。各プロジェクトは、入力データと出力データの仕様を詳細に記述したXML設定によって識別されます。

> The data used for labeling is stored in projects within Label Studio. Every project is identified by an XML configuration that details the specifications for input and output data.

テキスト形式で人間の入力を受け取り、テキストエリアに編集可能なLLMの応答を出力するプロジェクトを作成します：

> Create a project that takes human input in text format and outputs an editable LLM response in a text area:

```xml
<View>
<Style>
    .prompt-box {
        background-color: white;
        border-radius: 10px;
        box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);
        padding: 20px;
    }
</Style>
<View className="root">
    <View className="prompt-box">
        <Text name="prompt" value="$prompt"/>
    </View>
    <TextArea name="response" toName="prompt"
              maxSubmissions="1" editable="true"
              required="true"/>
</View>
<Header value="Rate the response:"/>
<Rating name="rating" toName="prompt"/>
</View>
```

1. Label Studioでプロジェクトを作成するには、「Create」ボタンをクリックしてください。

   > To create a project in Label Studio, click on the "Create" button.

2. `Project Name` フィールドに、`My Project` のようなプロジェクトの名前を入力してください。

   > Enter a name for your project in the "Project Name" field, such as `My Project`.

3. `ラベリング設定 > カスタムテンプレート`に移動し、上記で提供されたXML設定を貼り付けてください。

   > Navigate to `Labeling Setup > Custom Template` and paste the XML configuration provided above.





LabelStudioプロジェクトで入力LLMプロンプトと出力レスポンスを収集し、`LabelStudioCallbackHandler`を介して接続することができます。

> You can collect input LLM prompts and output responses in a LabelStudio project, connecting it via `LabelStudioCallbackHandler`:




In [ ]:
from langchain.callbacks import LabelStudioCallbackHandler
from langchain.llms import OpenAI

llm = OpenAI(
    temperature=0, callbacks=[LabelStudioCallbackHandler(project_name="My Project")]
)
print(llm("Tell me a joke"))

Label Studioで`My Project`を開いてください。プロンプト、レスポンス、そしてモデル名のようなメタデータが表示されることでしょう。

> In the Label Studio, open `My Project`. You will see the prompts, responses, and metadata like the model name.




## Collecting Chat model Dialogues | チャットモデルのダイアログ収集




LabelStudioを使用して、フルチャットダイアログを追跡し表示することができるだけでなく、最後の応答を評価し修正する能力もあります。

> You can also track and display full chat dialogues in LabelStudio, with the ability to rate and modify the last response:

1. Label Studioを開いて、「Create」ボタンをクリックしてください。

   > Open Label Studio and click on the "Create" button.

2. `Project Name`フィールドに、`New Project with Chat`のようなプロジェクトの名前を入力してください。

   > Enter a name for your project in the "Project Name" field, such as `New Project with Chat`.

3. ラベリング設定 > カスタムテンプレートに移動し、以下のXML設定を貼り付けてください：

   > Navigate to Labeling Setup > Custom Template and paste the following XML configuration:


```xml
<View>
<View className="root">
     <Paragraphs name="dialogue"
               value="$prompt"
               layout="dialogue"
               textKey="content"
               nameKey="role"
               granularity="sentence"/>
  <Header value="Final response:"/>
    <TextArea name="response" toName="dialogue"
              maxSubmissions="1" editable="true"
              required="true"/>
</View>
<Header value="Rate the response:"/>
<Rating name="rating" toName="dialogue"/>
</View>
```




In [ ]:
from langchain.callbacks import LabelStudioCallbackHandler
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

chat_llm = ChatOpenAI(
    callbacks=[
        LabelStudioCallbackHandler(
            mode="chat",
            project_name="New Project with Chat",
        )
    ]
)
llm_results = chat_llm(
    [
        SystemMessage(content="Always use a lot of emojis"),
        HumanMessage(content="Tell me a joke"),
    ]
)

Label Studioで、「New Project with Chat」を開き、作成されたタスクをクリックすると、ダイアログ履歴が表示され、レスポンスの編集や注釈ができます。

> In Label Studio, open "New Project with Chat". Click on a created task to view dialog history and edit/annotate responses.




## Custom Labeling Configuration | カスタムラベリング設定




LabelStudioのデフォルトのラベリング設定を変更して、レスポンスの感情、関連性、そして[他の様々なタイプのアノテーターのフィードバック](https://labelstud.io/tags/)を追加することができます。

> You can modify the default labeling configuration in LabelStudio to add more target labels like response sentiment, relevance, and many [other types annotator's feedback](https://labelstud.io/tags/).

新しいラベリング設定はUIから追加できます：`Settings > Labeling Interface`へ移動し、感情分析のための`Choices`や関連性評価のための`Rating`などの追加タグを含むカスタム設定を行ってください。どの設定にもLLMの応答を表示するために[`TextArea`タグ](https://labelstud.io/tags/textarea)が含まれている必要があることに注意してください。

> New labeling configuration can be added from UI: go to `Settings > Labeling Interface` and set up a custom configuration with additional tags like `Choices` for sentiment or `Rating` for relevance. Keep in mind that [`TextArea` tag](https://labelstud.io/tags/textarea) should be presented in any configuration to display the LLM responses.

あるいは、プロジェクト作成前の初回の呼び出し時にラベリング設定を指定することもできます：

> Alternatively, you can specify the labeling configuration on the initial call before project creation:




In [ ]:
ls = LabelStudioCallbackHandler(
    project_config="""
<View>
<Text name="prompt" value="$prompt"/>
<TextArea name="response" toName="prompt"/>
<TextArea name="user_feedback" toName="prompt"/>
<Rating name="rating" toName="prompt"/>
<Choices name="sentiment" toName="prompt">
    <Choice value="Positive"/>
    <Choice value="Negative"/>
</Choices>
</View>
"""
)

プロジェクトが存在しない場合は、指定されたラベリング設定で新たに作成されることに注意してください。

> Note that if the project doesn't exist, it will be created with the specified labeling configuration.




## Other parameters | その他のパラメーター




`LabelStudioCallbackHandler`は、いくつかのオプショナルパラメーターを受け入れます：

> The `LabelStudioCallbackHandler` accepts several optional parameters:

* **api\_key** - Label StudioのAPIキーです。環境変数 `LABEL_STUDIO_API_KEY` を上書きします。

  > **api\_key** - Label Studio API key. Overrides environmental variable `LABEL_STUDIO_API_KEY`.

* **url** - Label StudioのURL。`LABEL_STUDIO_URL`を上書きします。デフォルトは`http://localhost:8080`です。

  > **url** - Label Studio URL. Overrides `LABEL_STUDIO_URL`, default `http://localhost:8080`.

* **project\_id** - 既存のLabel StudioプロジェクトIDです。`LABEL_STUDIO_PROJECT_ID`を上書きします。このプロジェクトにデータを保存します。

  > **project\_id** - Existing Label Studio project ID. Overrides `LABEL_STUDIO_PROJECT_ID`. Stores data in this project.

* **project\_name** - プロジェクトIDが指定されていない場合のプロジェクト名です。新しいプロジェクトを作成します。デフォルトは現在の日付を用いて`"LangChain-%Y-%m-%d"`という形式でフォーマットされます。

  > **project\_name** - Project name if project ID not specified. Creates a new project. Default is `"LangChain-%Y-%m-%d"` formatted with the current date.

* **project\_config** - [カスタムラベリング設定](#custom-labeling-configuration)

  > **project\_config** - [custom labeling configuration](#custom-labeling-configuration)

* **モード**: このショートカットを使用して、最初からターゲットの設定を作成します：

  > **mode**: use this shortcut to create target configuration from scratch:
  
* "prompt" - 一つのプロンプトに対して一つのレスポンスを返します。デフォルト設定です。

    > `"prompt"` - Single prompt, single response. Default.

  * "chat" - 複数ターンのチャットモード。

    > `"chat"` - Multi-turn chat mode.



